In [1]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
sns.set()

In [2]:
import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.utils.data import random_split, Dataset, DataLoader

In [3]:
datasets = np.load("final set.npz")
datasets.allow_pickle = True
my_mapping = datasets["use_word2idx"]
dataset = datasets["dataset"]

dufu = torch.from_numpy(dataset.item()["dufu"])
sushi = torch.from_numpy(dataset.item()["sushi"])

label_dufu = torch.full((len(dufu), ), fill_value=0)
label_sushi = torch.full((len(sushi), ), fill_value=1)

final_label = torch.cat((label_dufu, label_sushi), dim=0)
final_dataset = torch.cat((dufu, sushi), dim=0)

final_label = final_label.type(torch.LongTensor)

In [4]:
dataset.item()["sushi"]

array([[   0,    0,    0, ..., 2456, 3507, 4682],
       [   0,    0,    0, ..., 1447, 3507, 4682],
       [   0,    0,    0, ..., 3166, 3507, 4682],
       ...,
       [   0,    0,    0, ..., 3124, 3507, 4682],
       [   0,    0,    0, ...,  216, 3507, 4682],
       [   0,    0,    0, ..., 4316, 3507, 4682]])

In [5]:
word2idx = datasets["word2idx"]
idx2word = datasets["idx2word"]

In [6]:
final_label.shape

torch.Size([2300])

In [7]:
final_dataset.shape

torch.Size([2300, 1210])

In [8]:
dataset_train, dataset_test, label_train, label_test = train_test_split(final_dataset,
                                                            final_label, test_size=0.1,
                                                                       random_state=1)

In [9]:
dataset_test.shape

torch.Size([230, 1210])

In [10]:
dataset_train.shape

torch.Size([2070, 1210])

### Dataset

In [11]:
class MyDataset(Dataset):
    
    def __init__(self, dataset, label):
        
        self.datasets = dataset
        self.labels = label

    def __getitem__(self, idx):
        return self.datasets[idx], self.labels[idx]

    def __len__(self):
        return len(self.labels)


### Model

In [12]:
class PoemClassifier(nn.Module):
    
    def __init__(self, words_num, embedding_size, hidden_size, classes, num_layers,
                    batch_size, sequence_length):
        super(PoemClassifier, self).__init__()
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.words_num = words_num
        self.sequence_length = sequence_length
        self.emb = nn.Embedding(words_num, embedding_size)
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, hidden=None):
        batch_size, sequence_length = x.shape # x batch_size, sequence_length
        if hidden is None:
            h, c = self.init_hidden(x, batch_size)
        else:
            h. c = hidden
        out = self.emb(x) # batch_size, sequence_length, embedding_size
        out, hidden = self.LSTM(out, (h, c)) # batch_size, sequence_length, hidden_size
        out = out[:, -1, :]# batch_size, last sequence, hidden_size
        out = self.fc1(out)
        return out, hidden

    def init_hidden(self, ipt, batch_size):
        h = ipt.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
        c = ipt.data.new(self.num_layers, batch_size, self.hidden_size).fill_(0).float()
        h = Variable(h)
        c = Variable(c)
        return (h, c)


In [13]:
len(dataset_train[1])

1210

### trainer

In [14]:
batch_size = 32
epoch = 40
model = PoemClassifier(len(my_mapping), 128, 128, 2, 2, batch_size, 300)
optimizer = optim.Adam(model.parameters(), lr=5e-3, weight_decay=0.003)
criterion = nn.CrossEntropyLoss()
model = model.cuda()
datasets = MyDataset(dataset_train, label_train)
data_loader = DataLoader(dataset=datasets, batch_size=batch_size, shuffle=True, drop_last=True, 
                                    num_workers=4)

In [15]:
model = model.cuda()
dataset_test = dataset_test.cuda()
label_test = label_test.cuda()
for e in range(1, 1 + epoch):
    for idx, item in enumerate(data_loader):
        data, labels = item
        data = data.cuda()
        labels = labels.cuda()
        h = None
        if idx == 0:
            out, h = model(data)
        else:
            out, h = model(data)
        loss = criterion(out, labels)
        p = (torch.sum(torch.max(out, dim=1)[1] == labels).item()) / batch_size
        acc = sum(torch.max(model(dataset_test.detach())[0], dim=1)[1] == label_test).item() / len(dataset_test)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch [{e}/{epoch}] step [{idx + 1}/{len(data_loader)}] loss = {loss.item()} accuracy = {p} val_acc = {round(acc, 4)}")

In [16]:
torch.save(model.state_dict(), "PoemClassify.pth")